# NLP Demo 

## Set up global variables and random_state

In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from datetime import datetime
from opendataval.util import set_random_state

device = torch.device("mps")
random_state = set_random_state(10)
date = datetime.now().strftime("%m-%d_%H:%M")

## Choose Experiment parameters

In [ ]:
from opendataval.dataloader import mix_labels
noise_rate=.1
nlp_datasets = ["bbc", "imdb"]
dataset_name = nlp_datasets[1]

nlp_models = ['bert']

dataset_name = nlp_datasets[0]
train_count, valid_count, test_count = 100, 50, 50
add_noise_func = mix_labels
noise_rate = 0.1
noise_kwargs = {'noise_rate': noise_rate }


model_name = nlp_models[0]
train_kwargs = {"epochs": 2, "batch_size": 50}
metric_name = "accuracy"

## Get ExperimentMediator without specifying DataEvaluators

In [ ]:
from opendataval.evaluator import ExperimentMediator

exper_med = ExperimentMediator.model_factory_setup(
    dataset_name=dataset_name,
    force_download=False,
    train_count=train_count,
    valid_count=valid_count,
    test_count=test_count,
    add_noise_func=add_noise_func,
    noise_kwargs=noise_kwargs,
    random_state=random_state,
    model_name=model_name,
    device=device,
    train_kwargs=train_kwargs,
    metric_name=metric_name
)

## Data Evaluators

#### Lots of imports for the many Data Evaluators

In [ ]:
from opendataval.dataval.ame import AME
from opendataval.dataval.influence import InfluenceFunctionEval
from opendataval.dataval.oob import DataOob
from opendataval.dataval import LeaveOneOut
from opendataval.dataval import BetaShapley, DataShapley
from opendataval.dataval import DataBanzhaf

#### Set up a series of data evaluators

In [ ]:
data_evaluators = [
    AME(num_models=1500, random_state=random_state),
    DataOob(random_state=random_state),  # 1000 samples
    # DVRL(rl_epochs=3000, random_state=random_state, device=device),  # DVRL requires tensor inputs
    LeaveOneOut(random_state=random_state),
    InfluenceFunctionEval(5000, random_state=random_state),
    DataBanzhaf(5000, random_state=random_state),
    BetaShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
    DataShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
]

In [ ]:
exper_med = exper_med.compute_data_values(data_evaluators=data_evaluators)

## Running experiments on the data values

In [ ]:
from opendataval.evaluator.exper_methods import remove_high_low, increasing_bin_removal, noisy_detection, discover_corrupted_sample, save_dataval

# Saving the results
import os
output_dir = f"tmp/{dataset_name}/{date}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Removing high values and low values

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(remove_high_low, include_train=True, col=2)
df_resp.to_csv(f"{output_dir}/remove_high_low.csv")

### Increasing bin removal

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(increasing_bin_removal, include_train=True, col=2)
df_resp.to_csv(f"{output_dir}/increasing_bin_removal.csv")

#### Noisy Detection

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp = exper_med.evaluate(noisy_detection)
df_resp.to_csv(f"{output_dir}/noisy_detection.csv")

#### Discover corrupted sample

In [ ]:
df_resp, fig = exper_med.plot(discover_corrupted_sample, col=2)
df_resp.to_csv(f"{output_dir}/discover_corrupted_sample.csv")

#### Save data values

In [ ]:
df_resp = exper_med.evaluate(save_dataval)
df_resp.to_csv(f"{output_dir}/save_dataval.csv")